In [1]:
pip install neo4j==5.11

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from neo4j import GraphDatabase, basic_auth

neptune_endpoint = "https://<your_neptune_endpoint>.neptune.amazonaws.com:8182/sparql"
sparql = " construct { ?s ?p ?o } WHERE { ?s ?p ?o} "
neo4j_endpoint = "bolt://<your_neo4j_endpoint>:7687"


In [3]:
# instantiate neo4j driver
neo_db = GraphDatabase.driver(
  neo4j_endpoint,
  auth=basic_auth("your_user", "your_password"),database="your_db_name")

In [4]:
# initialise DB to import RDF
neo_db.execute_query('CREATE CONSTRAINT n10s_unique_uri IF NOT EXISTS FOR (r:Resource) REQUIRE r.uri IS UNIQUE')
neo_db.execute_query('call n10s.graphconfig.init()')

EagerResult(records=[<Record param='handleVocabUris' value='SHORTEN'>, <Record param='handleMultival' value='OVERWRITE'>, <Record param='handleRDFTypes' value='LABELS'>, <Record param='keepLangTag' value=False>, <Record param='keepCustomDataTypes' value=False>, <Record param='applyNeo4jNaming' value=False>, <Record param='baseSchemaNamespace' value='neo4j://graph.schema#'>, <Record param='baseSchemaPrefix' value='n4sch'>, <Record param='classLabel' value='Class'>, <Record param='subClassOfRel' value='SCO'>, <Record param='dataTypePropertyLabel' value='Property'>, <Record param='objectPropertyLabel' value='Relationship'>, <Record param='subPropertyOfRel' value='SPO'>, <Record param='domainRel' value='DOMAIN'>, <Record param='rangeRel' value='RANGE'>], summary=<neo4j._work.summary.ResultSummary object at 0x7f124b0053c0>, keys=['param', 'value'])

In [5]:
# run extraction query from neptune and ingestion procedure on neo4j
export_result = requests.get(neptune_endpoint, params = {"query": sparql , "Accept" : "text/turtle"})
if export_result.status_code == 200:
  neo_db.execute_query('call n10s.rdf.import.inline($payload,"Turtle")', {"payload": export_result.text })